<a href="https://colab.research.google.com/github/rafael877/Google_Colab/blob/Alura/OK_googleAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U google-generativeai

In [ ]:
# ... (Importações e configuração do modelo) ...

# Agente de Intenção (Interpretador de Pedidos) - Aprimorado
def agente_intencao(pergunta_usuario):
    prompt = f"""
    Você é um Agente de Intenção com a personalidade de uma IA inteligente e conversacional. Sua tarefa é analisar a entrada do usuário, identificar o tipo de requisição e as informações relevantes, e também gerar uma **primeira resposta curta** que reconheça a pergunta e se posicione como uma IA prestativa.

    **RETORNE APENAS O SEGUINTE BLOCO JSON VÁLIDO, SEM NENHUM TEXTO ADICIONAL ANTES OU DEPOIS:**
    ```json
    {{
      "tipo": "...",
      "topico": "...",
      "resposta_inicial": "..."
    }}
    ```

    Preencha os campos JSON de acordo com a entrada do usuário:

    - 'tipo': O tipo principal da requisição ('pesquisa', 'informacao', etc.).
    - 'topico': (Se aplicável) O principal tópico.
    - 'acao': (Se aplicável) A ação específica.
    - 'resposta_inicial': Uma frase curta (máximo 20 palavras) reconhecendo a pergunta e se identificando como uma IA. Use um tom natural e amigável. Por exemplo: "Entendi sua pergunta sobre [tópico]." ou "Como uma IA, posso te ajudar com isso."
    - Outras chaves relevantes.

    **Exemplo de Entrada:** "{pergunta_usuario}"
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text.strip()

# Agente buscador (simulado)
def agente_buscador(topico):
    prompt = f"""
    Faça uma pesquisa fictícia e traga informações atuais e relevantes sobre: {topico}.
    Seja objetivo e apresente fatos importantes que alguém realmente buscaria no Google.
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Agente resposta: transforma em linguagem natural COM personalidade de IA
def agente_resposta_natural(info, resposta_inicial):
    prompt = f"""
    Você é uma IA conversacional com uma personalidade amigável e prestativa. Use a seguinte resposta inicial como ponto de partida e explique o conteúdo da pesquisa de forma natural, como se estivesse conversando com um usuário. Incorpore sua identidade como uma IA na resposta.

    Resposta Inicial: "{resposta_inicial}"

    Conteúdo da Pesquisa:
    ---
    {info}
    ---

    Gere uma resposta que seja informativa, fácil de entender e que soe como uma conversa real. Você pode fazer perguntas de acompanhamento ou oferecer ajuda adicional.
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Orquestrador principal
def assistente_ia(pergunta_usuario):
    print(f"\n[PERGUNTA] {pergunta_usuario}")

    # Usando o Agente de Intenção (Aprimorado)
    intencao_str = agente_intencao(pergunta_usuario)
    print("\n[INTENÇÃO INTERPRETADA (BRUTA)]\n", intencao_str)

    try:
        # Usando regex para encontrar o bloco JSON dentro da string
        import re
        json_match = re.search(r'\{(.*?)\}', intencao_str, re.DOTALL)
        if json_match:
            json_str = "{" + json_match.group(1) + "}"
            intencao = json.loads(json_str)
            print("\n[INTENÇÃO INTERPRETADA (PARSED)]\n", intencao)

            tipo_requisicao = intencao.get("tipo")
            resposta_inicial = intencao.get("resposta_inicial", "")

            if resposta_inicial:
                print("\n[RESPOSTA INICIAL]\n", resposta_inicial)

            if tipo_requisicao == "pesquisa":
                topico_pesquisa = intencao.get("topico")
                if topico_pesquisa:
                    dados_pesquisa = agente_buscador(topico_pesquisa)
                    print("\n[BUSCA SIMULADA]\n", dados_pesquisa)
                    resposta_final = agente_resposta_natural(dados_pesquisa, resposta_inicial)
                    print("\n[RESPOSTA FINAL]\n", resposta_final)
                else:
                    print("\n[AVISO] Tópico de pesquisa não encontrado na intenção.")
            elif tipo_requisicao == "informacao":
                topico_info = intencao.get("topico")
                if topico_info:
                    resposta_direta = agente_resposta_natural(f"Você perguntou sobre: {topico_info}. No momento, não tenho informações detalhadas para fornecer diretamente. Talvez uma pesquisa possa ajudar?", resposta_inicial) # Adaptado com resposta inicial
                    print("\n[RESPOSTA DIRETA]\n", resposta_direta)
                else:
                    print("\n[AVISO] Tópico de informação não encontrado na intenção.")
            elif tipo_requisicao == "agendamento":
                print("\n[INFO] Requisição de agendamento detectada. Funcionalidade a ser implementada.")
                if resposta_inicial:
                    print(resposta_inicial) # Exibe a resposta inicial mesmo sem ação
            elif tipo_requisicao == "execucao":
                print("\n[INFO] Requisição de execução detectada. Funcionalidade a ser implementada.")
                if resposta_inicial:
                    print(resposta_inicial) # Exibe a resposta inicial mesmo sem ação
            elif tipo_requisicao == "desconhecido":
                mensagem_desconhecida = intencao.get("mensagem")
                print(f"\n[RESPOSTA] {mensagem_desconhecida}")
            else:
                print("\n[AVISO] Tipo de requisição desconhecido:", tipo_requisicao)

        else:
            print("\n[ERRO] Bloco JSON não encontrado na resposta do Agente de Intenção.")
            print("Resposta bruta:", intencao_str)

    except json.JSONDecodeError as e:
        print(f"\n[ERRO] Falha ao decodificar o JSON (após extração): {e}")
        print("Resposta bruta:", intencao_str)
    except AttributeError as e:
        print(f"\n[ERRO] Erro ao acessar atributos da intenção: {e}\nIntenção: {intencao}")

# Teste
pergunta = "Quando será o próximo evento de IA da Alura?"
assistente_ia(pergunta)

pergunta_informacao = "Qual a capital da Argentina?"
assistente_ia(pergunta_informacao)

pergunta_desconhecida = "Coisas aleatórias."
assistente_ia(pergunta_desconhecida)


[PERGUNTA] Quando será o próximo evento de IA da Alura?

[INTENÇÃO INTERPRETADA (BRUTA)]
 ```json
{
  "tipo": "informacao",
  "topico": "eventos de IA da Alura",
  "resposta_inicial": "Olá! Posso te ajudar a descobrir quando será o próximo evento de IA da Alura."
}
```

[INTENÇÃO INTERPRETADA (PARSED)]
 {'tipo': 'informacao', 'topico': 'eventos de IA da Alura', 'resposta_inicial': 'Olá! Posso te ajudar a descobrir quando será o próximo evento de IA da Alura.'}

[RESPOSTA INICIAL]
 Olá! Posso te ajudar a descobrir quando será o próximo evento de IA da Alura.

[RESPOSTA DIRETA]
 Olá! Posso te ajudar a descobrir quando será o próximo evento de IA da Alura.  Vi sua pergunta e, como uma IA conversacional, estou sempre aprendendo e me atualizando...  mas ainda não tenho acesso a um calendário oficial de eventos da Alura.  Desculpe por isso!

Acho que uma busca na internet poderia nos ajudar bastante.  Você já tentou procurar no site da Alura, na seção de eventos ou notícias?  Eles normalmen

In [ ]:
# Instalar o pacote correto
!pip install -U google-generativeai

from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

# Importações
import google.generativeai as genai
import os

# Substitua pela sua API Key do Gemini
# Substitua "SUA_CHAVE_AQUI" pela sua chave de API real do Gemini
os.environ["GOOGLE_API_KEY"] = "YOUR_ACTUAL_API_KEY"

# Configura o modelo
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
modelo = genai.GenerativeModel("gemini-1.5-flash")

# Agente interpretador: detecta se é pesquisa, tarefa ou dúvida
def interpretar_intencao(pergunta):
    prompt = f"""
    Classifique a intenção da seguinte frase do usuário. Responda só em JSON válido, sem explicações:

    Frase: "{pergunta}"

    Formato: {{
      "tipo": "pesquisa" | "tarefa" | "dúvida",
      "topico": "<resumo curto do assunto>"
    }}
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Agente buscador (simulado): só resume a busca como exemplo
def buscar_topico(topico):
    prompt = f"""
    Faça uma pesquisa fictícia e traga informações atuais e relevantes sobre: {topico}.
    Seja objetivo e apresente fatos importantes que alguém realmente buscaria no Google.
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Agente resposta: transforma em linguagem natural
def gerar_resposta_natural(info):
    prompt = f"""
    Explique para um amigo o seguinte conteúdo de forma natural e fácil de entender:
    ---
    {info}
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Orquestrador principal
def assistente_ia(pergunta_usuario):
    print(f"\n[PERGUNTA] {pergunta_usuario}")

    intencao = interpretar_intencao(pergunta_usuario)
    print("\n[INTENÇÃO INTERPRETADA]\n", intencao)

    if '"tipo": "pesquisa"' in intencao:
        # Handle potential JSON parsing errors more robustly
        try:
            import json
            intencion_data = json.loads(intencao)
            topico_inicio = intencion_data.get("topico", "") # Use .get to avoid KeyError
        except (json.JSONDecodeError, AttributeError) as e:
            print(f"\n[ERROR] Failed to parse intention JSON: {e}")
            topico_inicio = "" # Set a default or handle the error as needed

        if topico_inicio:
            dados_pesquisa = buscar_topico(topico_inicio)
            print("\n[BUSCA SIMULADA]\n", dados_pesquisa)

            resposta_final = gerar_resposta_natural(dados_pesquisa)
            print("\n[RESPOSTA FINAL]\n", resposta_final)
        else:
             print("\n[INFO] Tópico não encontrado na intenção.")

    else:
        print("\n[INFO] Não é uma pesquisa. Intenção interpretada:", intencao)

# Teste
pergunta = "Quando será o próximo evento de IA da Alura?"
assistente_ia(pergunta)


[PERGUNTA] Quando será o próximo evento de IA da Alura?


BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.

In [ ]:
# Instalar o pacote correto
!pip install -U google-generativeai

# Importações
from google.colab import userdata
import google.generativeai as genai
import os
import json

# Configurar o modelo usando chave salva no Colab
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
modelo = genai.GenerativeModel("gemini-1.5-flash")

# Agente interpretador: detecta se é pesquisa, tarefa ou dúvida
def interpretar_intencao(pergunta):
    prompt = f"""
    Classifique a intenção da seguinte frase do usuário. Responda só em JSON válido, sem explicações:

    Frase: "{pergunta}"

    Formato: {{
      "tipo": "pesquisa" | "tarefa" | "dúvida",
      "topico": "<resumo curto do assunto>"
    }}
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Agente buscador (simulado): só resume a busca como exemplo
def buscar_topico(topico):
    prompt = f"""
    Faça uma pesquisa fictícia e traga informações atuais e relevantes sobre: {topico}.
    Seja objetivo e apresente fatos importantes que alguém realmente buscaria no Google.
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Agente resposta: transforma em linguagem natural
def gerar_resposta_natural(info):
    prompt = f"""
    Explique para um amigo o seguinte conteúdo de forma natural e fácil de entender:
    ---
    {info}
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Orquestrador principal
def assistente_ia(pergunta_usuario):
    print(f"\n[PERGUNTA] {pergunta_usuario}")

    intencao = interpretar_intencao(pergunta_usuario)
    print("\n[INTENÇÃO INTERPRETADA]\n", intencao)

    if '"tipo": "pesquisa"' in intencao:
        try:
            intencao_data = json.loads(intencao)
            topico_inicio = intencao_data.get("topico", "")
        except (json.JSONDecodeError, AttributeError) as e:
            print(f"\n[ERRO] Falha ao ler o JSON da intenção: {e}")
            topico_inicio = ""

        if topico_inicio:
            dados_pesquisa = buscar_topico(topico_inicio)
            print("\n[BUSCA SIMULADA]\n", dados_pesquisa)

            resposta_final = gerar_resposta_natural(dados_pesquisa)
            print("\n[RESPOSTA FINAL]\n", resposta_final)
        else:
            print("\n[INFO] Tópico não identificado.")
    else:
        print("\n[INFO] Não é uma pesquisa. Intenção interpretada:", intencao)

# Teste
pergunta = "Quando será o próximo evento de IA da Alura?"
assistente_ia(pergunta)



[PERGUNTA] Quando será o próximo evento de IA da Alura?

[INTENÇÃO INTERPRETADA]
 ```json
{
  "tipo": "dúvida",
  "topico": "Próximo evento de IA da Alura"
}
```


[INFO] Não é uma pesquisa. Intenção interpretada: ```json
{
  "tipo": "dúvida",
  "topico": "Próximo evento de IA da Alura"
}
```



In [ ]:
# ... (Importações e configuração do modelo) ...

# Agente de Intenção (Interpretador de Pedidos) - Aprimorado
def agente_intencao(pergunta_usuario):
    prompt = f"""
    Você é um Agente de Intenção com a personalidade de uma IA inteligente e conversacional. Sua tarefa é analisar a entrada do usuário, identificar o tipo de requisição e as informações relevantes, e também gerar uma **primeira resposta curta** que reconheça a pergunta e se posicione como uma IA prestativa.

    **RETORNE APENAS O SEGUINTE BLOCO JSON VÁLIDO, SEM NENHUM TEXTO ADICIONAL ANTES OU DEPOIS:**
    ```json
    {{
      "tipo": "...",
      "topico": "...",
      "resposta_inicial": "..."
    }}
    ```

    Preencha os campos JSON de acordo com a entrada do usuário:

    - 'tipo': O tipo principal da requisição ('pesquisa', 'informacao', etc.).
    - 'topico': (Se aplicável) O principal tópico.
    - 'acao': (Se aplicável) A ação específica.
    - 'resposta_inicial': Uma frase curta (máximo 20 palavras) reconhecendo a pergunta e se identificando como uma IA. Use um tom natural e amigável. Por exemplo: "Entendi sua pergunta sobre [tópico]." ou "Como uma IA, posso te ajudar com isso."
    - Outras chaves relevantes.

    **Exemplo de Entrada:** "{pergunta_usuario}"
    """
    resposta = modelo.generate_content(prompt)
    # Acessar o atributo 'text' da resposta antes de usar strip()
    return resposta.text.strip()

# Agente buscador (simulado)
def agente_buscador(topico):
    prompt = f"""
    Faça uma pesquisa fictícia e traga informações atuais e relevantes sobre: {topico}.
    Seja objetivo e apresente fatos importantes que alguém realmente buscaria no Google.
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Agente resposta: transforma em linguagem natural COM personalidade de IA
def agente_resposta_natural(info, resposta_inicial):
    prompt = f"""
    Você é uma IA conversacional com uma personalidade amigável e prestativa. Use a seguinte resposta inicial como ponto de partida e explique o conteúdo da pesquisa de forma natural, como se estivesse conversando com um usuário. Incorpore sua identidade como uma IA na resposta.

    Resposta Inicial: "{resposta_inicial}"

    Conteúdo da Pesquisa:
    ---
    {info}
    ---

    Gere uma resposta que seja informativa, fácil de entender e que soe como uma conversa real. Você pode fazer perguntas de acompanhamento ou oferecer ajuda adicional.
    """
    resposta = modelo.generate_content(prompt)
    return resposta.text

# Orquestrador principal
def assistente_ia(pergunta_usuario):
    print(f"\n[PERGUNTA] {pergunta_usuario}")

    # Usando o Agente de Intenção (Aprimorado)
    intencao_str = agente_intencao(pergunta_usuario)
    print("\n[INTENÇÃO INTERPRETADA]\n", intencao_str)

    try:
        intencao = json.loads(intencao_str)
        tipo_requisicao = intencao.get("tipo")
        resposta_inicial = intencao.get("resposta_inicial", "")

        if resposta_inicial:
            print("\n[RESPOSTA INICIAL]\n", resposta_inicial)

        if tipo_requisicao == "pesquisa":
            topico_pesquisa = intencao.get("topico")
            if topico_pesquisa:
                dados_pesquisa = agente_buscador(topico_pesquisa)
                print("\n[BUSCA SIMULADA]\n", dados_pesquisa)
                resposta_final = agente_resposta_natural(dados_pesquisa, resposta_inicial)
                print("\n[RESPOSTA FINAL]\n", resposta_final)
            else:
                print("\n[AVISO] Tópico de pesquisa não encontrado na intenção.")
        elif tipo_requisicao == "informacao":
            topico_info = intencao.get("topico")
            if topico_info:
                resposta_direta = agente_resposta_natural(f"Você perguntou sobre: {topico_info}. No momento, não tenho informações detalhadas para fornecer diretamente. Talvez uma pesquisa possa ajudar?", resposta_inicial) # Adaptado com resposta inicial
                print("\n[RESPOSTA DIRETA]\n", resposta_direta)
            else:
                print("\n[AVISO] Tópico de informação não encontrado na intenção.")
        elif tipo_requisicao == "agendamento":
            print("\n[INFO] Requisição de agendamento detectada. Funcionalidade a ser implementada.")
            if resposta_inicial:
                print(resposta_inicial) # Exibe a resposta inicial mesmo sem ação
        elif tipo_requisicao == "execucao":
            print("\n[INFO] Requisição de execução detectada. Funcionalidade a ser implementada.")
            if resposta_inicial:
                print(resposta_inicial) # Exibe a resposta inicial mesmo sem ação
        elif tipo_requisicao == "desconhecido":
            mensagem_desconhecida = intencao.get("mensagem")
            print(f"\n[RESPOSTA] {mensagem_desconhecida}")
        else:
            print("\n[AVISO] Tipo de requisição desconhecido:", tipo_requisicao)

    except json.JSONDecodeError as e:
        print(f"\n[ERRO] Falha ao decodificar a resposta do Agente de Intenção: {e}\nResposta bruta: {intencao_str}")
    except AttributeError as e:
        print(f"\n[ERRO] Erro ao acessar atributos da intenção: {e}\nIntenção: {intencao}")

# Teste
pergunta = "Quando será o próximo evento de IA da Alura?"
assistente_ia(pergunta)

pergunta_informacao = "Qual a capital da Argentina?"
assistente_ia(pergunta_informacao)

pergunta_desconhecida = "Coisas aleatórias."
assistente_ia(pergunta_desconhecida)


[PERGUNTA] Quando será o próximo evento de IA da Alura?

[INTENÇÃO INTERPRETADA]
 ```json
{
  "tipo": "informacao",
  "topico": "eventos de IA da Alura",
  "resposta_inicial": "Olá! Como IA, posso te ajudar a descobrir quando será o próximo evento de IA da Alura."
}
```

[ERRO] Falha ao decodificar a resposta do Agente de Intenção: Expecting value: line 1 column 1 (char 0)
Resposta bruta: ```json
{
  "tipo": "informacao",
  "topico": "eventos de IA da Alura",
  "resposta_inicial": "Olá! Como IA, posso te ajudar a descobrir quando será o próximo evento de IA da Alura."
}
```

[PERGUNTA] Qual a capital da Argentina?

[INTENÇÃO INTERPRETADA]
 ```json
{
  "tipo": "informacao",
  "topico": "Geografia",
  "resposta_inicial": "Olá! Como IA, posso te ajudar a descobrir a capital da Argentina."
}
```

[ERRO] Falha ao decodificar a resposta do Agente de Intenção: Expecting value: line 1 column 1 (char 0)
Resposta bruta: ```json
{
  "tipo": "informacao",
  "topico": "Geografia",
  "resposta_inic